In [90]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Load in the scouting df from the ``get_szn_data.ipynb`` output

In [91]:
szn = '2024-25'
df = pd.read_csv(f'data/scouting/scouting_{szn}.csv')
df.head()

,name,prev_position,team,xP,xA,xG,xGI,A,G,GI,bonus,bps,minutes,total_points,prev_price,GW,current_price,current_position,current_team
0,Jack Hinshelwood,MID,Brighton,1.5,0.00,0.00,0.00,0,0,0,0,0,0,0,45,1,50.0,MID,Brighton
1,Jadon Sancho,MID,Man Utd,3.0,0.05,0.00,0.05,0,0,0,0,4,22,1,70,1,65.0,MID,Man Utd
2,Vitaly Janelt,MID,Brentford,2.1,0.01,0.02,0.03,0,0,0,0,6,90,2,55,1,50.0,MID,Brentford
3,Jack Grealish,MID,Man City,4.1,0.00,0.00,0.00,0,0,0,0,0,0,0,75,1,65.0,MID,Man City
4,Emil Krafth,DEF,Newcastle,0.0,0.00,0.00,0.00,0,0,0,0,0,0,0,45,1,45.0,DEF,Newcastle


In [92]:
df['current_team'].unique()

array(['Brighton', 'Man Utd', 'Brentford', 'Man City', 'Newcastle',
       'Wolves', 'Liverpool', 'Chelsea', 'Spurs', 'Aston Villa',
       "Nott'm Forest", 'Arsenal', 'Fulham', 'Everton', 'Crystal Palace',
       'Bournemouth', 'West Ham', 'Ipswich', 'Southampton', 'Leicester'],
      dtype=object)

## Select a gamweek window (optional)

Let's select only the last half of the season, as that may be more indicative for current player performance. 

This collapses the the data across all selected GWs!

In [93]:
def select_gw_subset(df, gw_start, gw_end, min_mins):
    '''
    Function to select a subset of the FDR dataframe based on the gameweeks and teams you want to look at.
    
    Parameters:
    - df: merged GW DataFrame - each row is the stats for a player in a given GW.
    - gw_start: The starting gameweek (inclusive).
    - gw_end: The ending gameweek (inclusive).
    - min_mins: The minimum number of minutes a player must have played in the specified gameweeks.
    
    Returns:
    - A DataFrame subset based on the specified gameweeks - aggregated values are returned. 
    '''
    gws = np.arange(gw_start,gw_end+1)
    df = df[df['GW'].isin(gws)].reset_index(drop=True)
    df = df.drop(columns=['GW'])

    df = df.dropna()


    # df = df.groupby(by=['name', 'prev_position', 'current_position', 'team'], as_index=False).agg(
    df = df.groupby(by=['name', 'current_position', 'current_team'], as_index=False).agg(
        xP=('xP', 'sum'),
        xA=('xA', 'sum'),
        xG=('xG', 'sum'),
        xGI=('xGI', 'sum'),
        A=('A', 'sum'),
        G=('G', 'sum'),
        GI=('GI', 'sum'),
        mins=('minutes', 'sum'),
        points=('total_points', 'sum'),
        bonus=('bonus', 'sum'),
        prev_price=('prev_price', 'max'),
        now_price=('current_price', 'mean')
    )

    df = df[df['mins'] >= min_mins].reset_index(drop=True)

    # check that there are players that meet the criteria
    if len(df) == 0:
        raise ValueError(f"Criteria for GWs {gw_start}-{gw_end} and min minutes {min_mins} not met")

    df['now_price'] = df['now_price'].astype(int)
    df['prev_price'] = df['prev_price'].round(0).astype(int)

    df = df.rename(columns={'current_position': 'position', 'current_team': 'team'})
    
    return df


In [94]:
df_subset = select_gw_subset(df, 19, 38, min_mins=650)
df_subset.head()

,name,position,team,xP,xA,xG,xGI,A,G,GI,mins,points,bonus,prev_price,now_price
0,Aaron Wan-Bissaka,DEF,Man Utd,15.7,0.50,0.08,0.58,1,0,1,1165,20,0,44,45
1,Abdoulaye Doucouré,MID,Everton,37.9,0.55,3.08,3.63,1,1,2,1190,40,0,55,55
2,Adam Smith,DEF,Bournemouth,63.0,0.69,0.05,0.74,1,0,1,1579,54,4,44,45
3,Adam Wharton,MID,Crystal Palace,47.7,1.50,0.38,1.87,3,0,3,1296,50,4,50,50
4,Alejandro Garnacho,MID,Man Utd,92.5,2.08,5.36,7.44,6,6,12,1682,94,7,50,65


## Points per $

Let's add some informative columns

In [95]:
# points per minute
df_subset['ppm'] = df_subset['points'] / df_subset['mins']

# points per cost
df_subset['ppc'] = (df_subset['points'] / df_subset['now_price']).round(3)


Let's see which players give you the most bang for your buck - points per cost (ppc)

In [96]:
df_subset = df_subset.sort_values(by=['ppc'], ascending=False).reset_index(drop=True)
df_subset.head(10)

,name,position,team,xP,xA,xG,xGI,A,G,GI,mins,points,bonus,prev_price,now_price,ppm,ppc
0,Jordan Pickford,GK,Everton,100.7,0.25,0.00,0.25,0,0,0,1800,89,16,48,50,0.049444,1.780
1,Benjamin White,DEF,Arsenal,143.4,2.27,0.66,2.93,3,3,6,1668,115,7,61,65,0.068945,1.769
2,Kai Havertz,FWD,Arsenal,139.6,2.79,8.15,10.93,9,9,18,1535,130,15,76,80,0.084691,1.625
3,Mark Flekken,GK,Brentford,75.6,0.08,0.00,0.08,1,0,1,1890,73,11,47,45,0.038624,1.622
4,Phil Foden,MID,Man City,182.6,4.45,6.49,10.94,4,15,19,1526,154,23,85,95,0.100917,1.621
5,Cole Palmer,MID,Chelsea,192.0,4.90,11.87,16.77,9,16,25,1557,170,24,63,105,0.109184,1.619
6,Jean-Philippe Mateta,FWD,Crystal Palace,117.4,0.62,8.33,8.95,3,14,17,1651,121,17,51,75,0.073289,1.613
7,Jarrad Branthwaite,DEF,Everton,77.7,0.36,1.26,1.62,1,3,4,1766,80,5,45,50,0.045300,1.600
8,Bruno Guimarães Rodriguez Moura,MID,Newcastle,101.2,3.71,2.94,6.65,7,6,13,1772,103,12,58,65,0.058126,1.585
9,David Raya Martin,GK,Arsenal,106.5,0.02,0.00,0.02,0,0,0,1710,87,4,53,55,0.050877,1.582


## Overperforming and underperforming players

In [97]:
df_subset['xGI_GI_diff'] = df_subset['xGI'] - df_subset['GI']

Underperforming players - had a lot more expected goal involvements than they did 

In [98]:
df_subset = df_subset.sort_values(by=['xGI_GI_diff'], ascending=False).reset_index(drop=True)
df_subset.head(10)

,name,position,team,xP,xA,xG,xGI,A,G,GI,mins,points,bonus,prev_price,now_price,ppm,ppc,xGI_GI_diff
0,João Pedro Junqueira de Jesus,FWD,Brighton,38.5,2.59,6.87,9.46,2,4,6,1079,51,6,55,55,0.047266,0.927,3.46
1,James Tarkowski,DEF,Everton,57.6,1.56,1.52,3.08,0,0,0,1800,52,4,46,50,0.028889,1.040,3.08
2,Keane Lewis-Potter,MID,Brentford,45.2,1.31,4.73,6.04,1,2,3,1020,48,0,47,50,0.047059,0.960,3.04
3,Pascal Groß,MID,Brighton,75.7,5.25,3.68,8.93,5,1,6,1741,76,14,66,65,0.043653,1.169,2.93
4,Miguel Almirón Rejala,MID,Newcastle,23.1,1.65,0.78,2.43,0,0,0,684,19,0,62,60,0.027778,0.317,2.43
5,Manuel Akanji,DEF,Man City,93.8,1.09,1.15,2.24,0,0,0,1431,70,7,50,55,0.048917,1.273,2.24
6,Curtis Jones,MID,Liverpool,38.0,0.76,2.44,3.20,0,1,1,698,26,2,49,55,0.037249,0.473,2.20
7,Andreas Hoelgebaum Pereira,MID,Fulham,67.7,3.67,3.42,7.09,3,2,5,1392,64,4,53,55,0.045977,1.164,2.09
8,Mohamed Salah,MID,Liverpool,78.0,2.99,10.05,13.04,5,6,11,971,74,6,136,125,0.076210,0.592,2.04
9,Jack Harrison,MID,Everton,47.1,1.71,2.28,3.99,0,2,2,1265,47,2,55,55,0.037154,0.855,1.99


Overperforming players - had a lot more goal invovements than they did

In [99]:
df_subset = df_subset.sort_values(by=['xGI_GI_diff'], ascending=True).reset_index(drop=True)
df_subset.head(10)

,name,position,team,xP,xA,xG,xGI,A,G,GI,mins,points,bonus,prev_price,now_price,ppm,ppc,xGI_GI_diff
0,Ollie Watkins,FWD,Aston Villa,107.2,2.03,8.16,10.19,9,10,19,1643,118,15,90,90,0.071820,1.311,-8.81
1,Cole Palmer,MID,Chelsea,192.0,4.90,11.87,16.77,9,16,25,1557,170,24,63,105,0.109184,1.619,-8.23
2,Phil Foden,MID,Man City,182.6,4.45,6.49,10.94,4,15,19,1526,154,23,85,95,0.100917,1.621,-8.06
3,Jean-Philippe Mateta,FWD,Crystal Palace,117.4,0.62,8.33,8.95,3,14,17,1651,121,17,51,75,0.073289,1.613,-8.05
4,Rasmus Højlund,FWD,Man Utd,81.3,0.58,3.97,4.55,2,10,12,1218,89,14,72,70,0.073071,1.271,-7.45
5,Kai Havertz,FWD,Arsenal,139.6,2.79,8.15,10.93,9,9,18,1535,130,15,76,80,0.084691,1.625,-7.07
6,Anthony Gordon,MID,Newcastle,97.3,3.10,5.03,8.13,10,5,15,1500,92,6,64,75,0.061333,1.227,-6.87
7,Bruno Guimarães Rodriguez Moura,MID,Newcastle,101.2,3.71,2.94,6.65,7,6,13,1772,103,12,58,65,0.058126,1.585,-6.35
8,Declan Rice,MID,Arsenal,112.1,4.09,2.09,6.18,8,4,12,1662,100,10,55,65,0.060168,1.538,-5.82
9,Matheus Santos Carneiro Da Cunha,FWD,Wolves,48.2,1.20,4.71,5.91,4,7,11,934,67,9,58,65,0.071734,1.031,-5.09


## Top Bonus Points

In [100]:
df_subset = df_subset.sort_values(by=['bonus'], ascending=False).reset_index(drop=True)
df_subset.head(10)

,name,position,team,xP,xA,xG,xGI,A,G,GI,mins,points,bonus,prev_price,now_price,ppm,ppc,xGI_GI_diff
0,Cole Palmer,MID,Chelsea,192.0,4.90,11.87,16.77,9,16,25,1557,170,24,63,105,0.109184,1.619,-8.23
1,Phil Foden,MID,Man City,182.6,4.45,6.49,10.94,4,15,19,1526,154,23,85,95,0.100917,1.621,-8.06
2,Bukayo Saka,MID,Arsenal,128.0,5.15,10.17,15.31,4,11,15,1439,128,18,92,100,0.088951,1.280,0.31
3,Jean-Philippe Mateta,FWD,Crystal Palace,117.4,0.62,8.33,8.95,3,14,17,1651,121,17,51,75,0.073289,1.613,-8.05
4,Jordan Pickford,GK,Everton,100.7,0.25,0.00,0.25,0,0,0,1800,89,16,48,50,0.049444,1.780,0.25
5,Ollie Watkins,FWD,Aston Villa,107.2,2.03,8.16,10.19,9,10,19,1643,118,15,90,90,0.071820,1.311,-8.81
6,Kai Havertz,FWD,Arsenal,139.6,2.79,8.15,10.93,9,9,18,1535,130,15,76,80,0.084691,1.625,-7.07
7,Alexander Isak,FWD,Newcastle,113.7,1.61,12.58,14.19,4,14,18,1387,113,15,84,85,0.081471,1.329,-3.81
8,Chris Wood,FWD,Nott'm Forest,69.4,0.72,8.94,9.66,0,10,10,1253,84,15,50,60,0.067039,1.400,-0.34
9,Erling Haaland,FWD,Man City,130.9,1.01,14.62,15.63,3,13,16,1259,105,14,145,150,0.083400,0.700,-0.37
